In [1]:
import pandas as pd

In [2]:
pandas_df = pd.read_csv('C:/Users/hedin/Documents/pyspark-the correct way/notebooks/diabetes.csv')
pandas_df = pandas_df.loc[(pandas_df['Glucose'] != 0)
& (pandas_df['BloodPressure'] != 0)
& (pandas_df['BMI'] != 0),
['Pregnancies',
'Glucose',
'BloodPressure',
'BMI',
'DiabetesPedigreeFunction',
'Age',
'Outcome'
]]

In [3]:
#import the libs for the analisis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [5]:
#define the feature and target columns
x_cols = ['Pregnancies', 'Glucose', 'BloodPressure', 'BMI',
          'DiabetesPedigreeFunction', 'Age']
x = pandas_df[x_cols]
y = pandas_df["Outcome"]

In [6]:
#standarize the features using the standardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [7]:
#define a list of classifiers
classifiers = [LogisticRegression(), DecisionTreeClassifier(),
               RandomForestClassifier(), LinearSVC(max_iter=1500)]

In [15]:
#use a for-loop to perform ten-fold cross-validation
for classifier in classifiers:
    result = cross_val_score(
        classifier,
        x_scaled,
        y,
        cv=10,
        scoring="roc_auc"
    )
    print(
        f"{classifier.__class__.__name__} "
        f"mean: {result.mean():.2f}"
    )


LogisticRegression mean: 0.84
DecisionTreeClassifier mean: 0.68
RandomForestClassifier mean: 0.82
LinearSVC mean: 0.84
